# Azure MVAD Service Guide - Part I (Python SDK)

What follows is a guide via example on how to use Azure's MVAD Service. Make sure to check the service's documentation [here](https://learn.microsoft.com/en-us/azure/cognitive-services/anomaly-detector/overview), as well as the code provided [here](https://github.com/Azure-Samples/AnomalyDetector). Note that there are two approaches to using the MVAD Service: via the SDK and via the API. The former is presented in this notebook.

## Initial Setup

To use the MVAD Service we obviously need to create an [Anomaly Detector subscription](https://azure.microsoft.com/en-us/free/cognitive-services/), as well as create an [Anomaly Detector resource](https://ms.portal.azure.com/#create/Microsoft.CognitiveServicesAnomalyDetector). From the created resource, we need the endpoint's name, as well as the key. For safety, the key can be inserted into an Azure Key Vault and read from there by creating a [secret scope in Databricks](https://learn.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes). Additionally, a Storage Account is required, from where the data are drawn to perform anomaly detection. The MVAD Service (Managed Identity) must have the Storage Blob Data Reader role assigned to it in order to be able to draw data from the Storage Account.

For the purposes of the present demonstration, we create a new Container named `test-mvad` in the Storage Account present in the same resource group and send the [sample data](https://github.com/Azure-Samples/AnomalyDetector/blob/master/sampledata/multivariate/multivariate_sample_data.csv) that are going to be used as an example. The MVAD Service accepts data in both multi-table (one `.csv` file for each feature) and single-table (a single `.csv` file with multiple columns, one per feature plus the timestamp columns) forms, but we opt to work with the single-table approach.

In [ ]:
import time
import pandas as pd

The following library needs to be installed. Note that its installation can be attached to the startup of a specific cluster that runs the present notebook, or to a script that is activated whenever the cluster starts.

In [ ]:
#!pip install --upgrade azure-ai-anomalydetector

## Setup

First, we import the needed modules and then create a client for the anomaly detector service.

In [ ]:
from azure.ai.anomalydetector import AnomalyDetectorClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.anomalydetector.models import ModelInfo, AlignPolicy, AlignMode, FillNAMethod, MultivariateBatchDetectionOptions

In [ ]:
ad_name = "REDACTED"
ANOMALY_DETECTOR_ENDPOINT= f"https://{ad_name}.cognitiveservices.azure.com/"
# The key is stored in the Vault for safety
SUBSCRIPTION_KEY = dbutils.secrets.get(scope = "anomdet", key = "adkey")

# Setup the AD client
ad_client = AnomalyDetectorClient(ANOMALY_DETECTOR_ENDPOINT, AzureKeyCredential(SUBSCRIPTION_KEY))

## Basic Functionalities

With the client created, we can proceed to see some of the basic functionalities of the MVAD Service.

### Training

The first and perhaps most important one is the training of the model. This is done using the `train_multivariate_model()` method, which takes a `ModelInfo` object as its argument. A `ModelInfo` object is a custom `anomalydetector.models` object that corresponds to the training objective and result of an AD model, including its status, errors and diagnostics information. Its arguments can be seen in the next code cell, where one such object is defined to be used as the argument of the `train_multivariate_model()` method.

In [ ]:
storage_name = "REDACTED"
container_name = "test-mvad"
data_file_name = "sample.csv"

DATA_URL = f"https://{storage_name}.blob.core.windows.net/{container_name}/{data_file_name}"

train_start = "2021-02-18T13:50:00Z"
train_end = "2021-03-23T10:00:00Z"

train_body = ModelInfo(
        # required - source link to the input data to indicate an accessible Azure storage URI
        data_source = DATA_URL,
        # Data schema of input data source: OneTable or MultiTable. The default DataSchema is OneTable
        data_schema = "OneTable",
        # required - indicates the start time of training data, which should be date-time of ISO 8601 format
        start_time=train_start,
        # required - indicates the end time of training data, which should be date-time of ISO 8601 format
        end_time=train_end,
        # The display name of the model (max length is 24 characters).
        display_name="sample_model",
        # An optional field, indicating how many previous timestamps will be used to detect whether a timestamp is an anomaly or not
        sliding_window=200,
        # An optional field, indicating the manner to align multiple variables.
        # More info about the AlignPolicy method can be found below
        align_policy=AlignPolicy(
            align_mode=AlignMode.OUTER,
            fill_n_a_method=FillNAMethod.LINEAR,
            padding_value=0,
        ),
    )

Here is the documentation for the `AlignPolicy` method.

```
class AlignPolicy(_model_base.Model):
    An optional field, indicating the manner to align multiple variables.

    - align_mode: An optional field, indicating how to align different variables to the same
        time-range. Either Inner or Outer. Known values are: "Inner" and "Outer".
        
    - fill_n_a_method: An optional field, indicating how missing values will be filled. One of
        Previous, Subsequent, Linear, Zero, Fixed. Known values are: "Previous", "Subsequent",
        "Linear", "Zero", and "Fixed".
        
    - padding_value: An optional field. Required when fillNAMethod is Fixed.
```

Using the `train_body` parameter, we may proceed with the training of a MVAD model for the sample data previously uploaded in the corresponding container.

In [ ]:
# Start training a new model
ad_model = ad_client.train_multivariate_model(train_body)
# Get its ID
model_id = ad_model.model_id
print(f"A new AD model was created with ID: {model_id}. Please wait until the model is trained (status=READY).")

A new AD model was created with ID: c3a8c3bc-e917-11ed-bada-ee9e080c64ed. Please wait until the model is trained (status=READY).


At this point, the new model is being trained behind the shadows. By running the following cell's code, we can confirm this.

In [ ]:
model_status = ad_client.get_multivariate_model(model_id).model_info.status
print(f"Model status: {model_status}")

Model status: RUNNING


Instead of re-running this cell, we can define a loop that will perform the check for us and notify us once the model is truly ready. Once it is, we can also get some diagnostics.

In [ ]:
while True:
    # Perform a status request every 45 seconds, to see when the model is ready.
    model_status = ad_client.get_multivariate_model(model_id).model_info.status

    if model_status == 'READY':
        print("The model has been successfully trained.\n")
        break
    else:
        print("The model is still in training mode. Please wait...")
        epochs = ad_client.get_multivariate_model(model_id).model_info.diagnostics_info.model_state['epochIds']
        print("Here is a list of current number of training epochs:")
        print(epochs)
        time.sleep(45)

# Print diagnostics once the model is ready
minf = ad_client.get_multivariate_model(model_id).model_info

print('Training Info:\n')
display(pd.DataFrame(minf['diagnosticsInfo']['modelState']))
print("\nVariables Info:\n")
display(pd.DataFrame(minf['diagnosticsInfo']['variableStates']))
print(f"\nModel status: {minf['status']}")

The model is still in training mode. Please wait...
Here is a list of current number of training epochs:
[]
The model is still in training mode. Please wait...
Here is a list of current number of training epochs:
[10, 20, 30]
The model is still in training mode. Please wait...
Here is a list of current number of training epochs:
[10, 20, 30, 40, 50, 60, 70]
The model has been successfully trained.

Training Info:



epochIds,trainLosses,validationLosses,latenciesInSeconds
10,0.5775564798553076,0.8278671625312468,1.1147723197937012
20,0.5705596167328103,0.6265384489745301,1.344914436340332
30,0.5613522269496961,0.6976657031708204,1.0915250778198242
40,0.5525602697660881,0.7443105535077345,1.3077692985534668
50,0.5769657905080489,0.63325916886849,1.5008914470672607
60,0.5589256643184595,0.658607994743071,1.0671508312225342
70,0.5603698659023004,0.5808213519646559,1.1045498847961426
80,0.5614311744991157,0.6329505604070089,1.0595669746398926
90,0.5586518652604094,0.6206884510870074,1.143542766571045
100,0.5636797646459725,0.6026435065165602,1.1360013484954834



Variables Info:



variable,filledNARatio,effectiveCount,firstTimestamp,lastTimestamp
opticalLFiltered,0.0,4730,2021-02-18T13:50:00Z,2021-03-23T10:00:00Z
opticalRFiltered,0.0,4730,2021-02-18T13:50:00Z,2021-03-23T10:00:00Z
pumpPressure,0.0,4730,2021-02-18T13:50:00Z,2021-03-23T10:00:00Z
rotational,0.0,4730,2021-02-18T13:50:00Z,2021-03-23T10:00:00Z
vibrationHorizon,0.0,4730,2021-02-18T13:50:00Z,2021-03-23T10:00:00Z



Model status: READY


With the model at hand, we may proceed to see other operations available in the Azure MVAD service. Before doing so, it is worth noting that the selection of a model or its properties requires the model's ID. Nonetheless, we may be more familiar with model names instead of IDs. If we can guarantee that every model's name is unique, the following function can be used to get the model's ID by using its name.

In [ ]:
# This function should work as long as there is a unique name for each model, otherwise it only fetches the first result
def get_model_id(client, model_name):

    models = list(client.list_multivariate_models())

    models_df = pd.DataFrame([{'Model ID': x['modelId'], "Name" : x['modelInfo']['displayName']} for x in models])

    model_ids = models_df[models_df['Name']==model_name]['Model ID'].values
    if len(model_ids) == 1:
        model_id = model_ids[0]
    elif len(model_ids) == 0:
        print("No model with this name exists in the registry.")
        model_id = None
    else:
        print("Warning! There are more than one models with the given name.")
        print("Fetching the first among them.")
        model_id = model_ids[0]

    return model_id

Let's see the model ID for the model we just created and trained:

In [ ]:
model_id = get_model_id(ad_client, 'sample_model')
print(f"The ID of the newly trained model is {model_id}.")

The ID of the newly trained model is c3a8c3bc-e917-11ed-bada-ee9e080c64ed.


### Inference

The other main functionality is inference. In Azure MVAD there are two types of inference: batch and streaming, where each is self-explanatory. Streaming is oriented towards real-time problems, where each new point is classified as anomalous or not individually. For the purposes of our project, batch inference is the go-to, so we will focus on this in what follows. For more information about Streaming Inference you can refer [here](https://learn.microsoft.com/en-us/azure/cognitive-services/anomaly-detector/how-to/streaming-inference).

As far as Batch Inference is concerned, we first define a `MultivariateBatchDetectionOptions` object, that takes as arguments the options for the batch inference to be performed. Its arguments can be seen in the next code cell.

In [ ]:
storage_name = "REDACTED"
container_name = "test-mvad"
data_file_name = "sample.csv"

DATA_URL = f"https://{storage_name}.blob.core.windows.net/{container_name}/{data_file_name}"

inf_start = "2021-09-09T15:10:00Z"
inf_end = "2021-09-14T23:20:00Z"

batch_inference_body = MultivariateBatchDetectionOptions(
        # required - source link to the input data to indicate an accessible Azure storage URI
        # The data schema should be exactly the same with those used in the training phase
        data_source=DATA_URL,
        # required - used to specify the number of top contributed variables for one
        # anomalous timestamp in the response. The default number is 10.
        top_contributor_count=10,
        # required - indicates the start time of data for detection, which should be date-time of ISO 8601 format.
        start_time=inf_start,
        # required - indicates the end time of data for detection, which should be date-time of ISO 8601 format.
        end_time=inf_end
    )

In [ ]:
# Use the API to post the inference request
inf_request = ad_client.detect_multivariate_batch_anomaly(model_id, batch_inference_body)
result_id = inf_request.result_id
print(f"A batch inference is triggered with ID: {result_id}. Please refer to the following cells for the results.")

A batch inference is triggered with ID: 2b375f48-e918-11ed-b88a-1a5c9f1e1963. Please refer to the following cells for the results.


Similarly with the training process, the inference process might also take a while, depending on data types and sizes. We need to wait until the results are ready. Once they are, a different API call needs to be performed to obtain the results.

In [ ]:
while True:
    # Perform a status request every 15 seconds, to see when the results are ready.
    anomaly_results = ad_client.get_multivariate_batch_detection_result(result_id)

    if anomaly_results['summary']['status'] == 'READY':
        print("The results are ready.\n")
        break
    else:
        print("Please wait further for the inference results...")
        time.sleep(15)

# Print a summary when the results are ready.
print('Summary:\n')
display(pd.DataFrame(anomaly_results['summary']['variableStates']))

The results are ready.

Summary:



variable,filledNARatio,effectiveCount,firstTimestamp,lastTimestamp
opticalLFiltered,0.0,770,2021-09-09T15:10:00Z,2021-09-14T23:20:00Z
opticalRFiltered,0.0,770,2021-09-09T15:10:00Z,2021-09-14T23:20:00Z
pumpPressure,0.0,770,2021-09-09T15:10:00Z,2021-09-14T23:20:00Z
rotational,0.0,770,2021-09-09T15:10:00Z,2021-09-14T23:20:00Z
vibrationHorizon,0.0,770,2021-09-09T15:10:00Z,2021-09-14T23:20:00Z


The `anomaly_results` object now holds the results of the batch inference process, which are stored in a dict-like format in `anomaly_results['results']`. The results in a DataFrame format can be seen by running the following cell. Of course, a dataframe of results is not the best medium of serving them, which is why we will also proceed with a visualization thereof in the last notebook of this series.

In [ ]:
adf = pd.DataFrame([{'timestamp': x['timestamp'], **x['value']} for x in anomaly_results.results])
adf['timestamp'] = pd.to_datetime(adf['timestamp'], utc=True)
adf

,timestamp,isAnomaly,severity,score,interpretation
0,2021-09-09 15:10:00+00:00,True,0.073315,0.125975,"[{'variable': 'vibrationHorizon', 'contributio..."
1,2021-09-09 15:20:00+00:00,True,0.104373,0.179341,"[{'variable': 'vibrationHorizon', 'contributio..."
2,2021-09-09 15:30:00+00:00,True,0.123244,0.211768,"[{'variable': 'vibrationHorizon', 'contributio..."
3,2021-09-09 15:40:00+00:00,True,0.119344,0.205067,"[{'variable': 'opticalRFiltered', 'contributio..."
4,2021-09-09 15:50:00+00:00,True,0.149478,0.256845,"[{'variable': 'rotational', 'contributionScore..."
...,...,...,...,...,...
765,2021-09-14 22:40:00+00:00,False,0.000000,0.010069,[]
766,2021-09-14 22:50:00+00:00,False,0.000000,0.010069,[]
767,2021-09-14 23:00:00+00:00,False,0.000000,0.010069,[]
768,2021-09-14 23:10:00+00:00,False,0.000000,0.010069,[]


We can see that the results consist of the following fields:

- **isAnomaly**: the main field, indicating whether the given timestamp corresponds to an anomaly or not.
- **severity**: indicates the significance of the anomaly. The higher the severity, the more significant the anomaly is. A new threshold can be assigned so that anomalies are not found simply by checking the boolean `isAnomaly`, but also by checking if their severity crosses said threshold.
- **score**: raw anomaly, helps indicate the degree of abnormality as well. Notice that even non-anomalous points have a non-zero score, which is what makes this metric different from `severity`.
- **interpretation**: The interpretation result corresponds to a list where the contribution of each feature for the inference is given. This contribution is given as a score, as well as in the form of a `~anomalydetector.models.CorrelationChanges` type object. This object is basically a list of other features, for which the correlation between themselves and the studied feature has changed, as detected by the model.

### List available models

Another thing we can do with the Python SDK for the MVAD Service is get a list of the available models.

In [ ]:
def display_models(client):
    # The list_multivariate_models() method yields all the multivariate models that exist in the registry
    models = list(client.list_multivariate_models())
    # Cast the info we want to see in a dataframe
    models_df = pd.DataFrame([{'Model ID': x['modelId'], "Created" : x['createdTime'], "Last Updated" : x['lastUpdatedTime'], "Status" : x['modelInfo']['status'], "Name" : x['modelInfo']['displayName'], "Errors" : x['modelInfo']['errors'] if x['modelInfo']['errors'] else "None", "Variables Count" : len(x['modelInfo']['diagnosticsInfo']['variableStates'])} for x in models])
    # Display
    display(models_df)
    return

display_models(ad_client)

Model ID,Created,Last Updated,Status,Name,Errors,Variables Count
REDACTED,2023-05-02T18:32:59Z,2023-05-02T18:35:12Z,READY,sample_model,None,5
REDACTED,2022-12-13T17:47:32Z,2022-12-13T17:49:27Z,READY,MVADModel,None,3


Here we can see the model we trained on the sample data, as well as an older model.

### Delete model

Using the `delete_multivariate_model()` method, we may delete the model we trained from the registry, by providing its id as an argument. For example, we can get the id of the model we just trained and delete it from the registry.

In [ ]:
model_id = 'c3a8c3bc-e917-11ed-bada-ee9e080c64ed' # <- Insert the ID of the model to be deleted

ad_client.delete_multivariate_model(model_id)

If we now attempt to see again the list of available models, we will see that the model we trained has been successfully deleted.

In [ ]:
display_models(ad_client)

Model ID,Created,Last Updated,Status,Name,Errors,Variables Count
REDACTED,2022-12-13T17:47:32Z,2022-12-13T17:49:27Z,READY,MVADModel,None,3


This sums up the main functionalities of the MVAD Service using the Python SDK. Please refer to the next notebook for how to use the Service using the API, as well as the third notebook for visualization examples.